In [ ]:
# third party
import matplotlib.pyplot as plt
import numpy as np

# syft absolute
import syft as sy

## 1. DS logins to the datasite with the credentials created by the DO

In [ ]:
server = sy.orchestra.launch(name="mnist-torch-datasite", dev_mode=True)
ds_client = server.login(email="sheldon@caltech.edu", password="changethis")

### Inspect the datasets on the datasite

In [ ]:
datasets = ds_client.datasets.get_all()
assert len(datasets) == 1
datasets

In [ ]:
assets = datasets[0].assets
assert len(assets) == 2
assets

In [ ]:
training_images = assets[0]
training_images

In [ ]:
training_labels = assets[1]
training_labels

#### The DS can not access the real data

In [ ]:
assert training_images.data is None
training_labels.data

#### The DS can only access the mock data, which is some random noise

In [ ]:
mock_images = training_images.mock
plt.imshow(np.reshape(mock_images[0], (28, 28)))

#### We need the pointers to the mock data to construct a `syft` function (later in the notebook)

In [ ]:
mock_images_ptr = training_images.pointer
mock_images_ptr

In [ ]:
type(mock_images_ptr)

In [ ]:
mock_labels = training_labels.mock
mock_labels_ptr = training_labels.pointer
mock_labels_ptr

## 2. The DS prepare the training code and experiment on the mock data

In [ ]:
def mnist_3_linear_layers_torch(mnist_images, mnist_labels):
    # third party
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset

    # Convert NumPy arrays to PyTorch tensors
    images_tensor = torch.tensor(mnist_images, dtype=torch.float32)
    labels_tensor = torch.tensor(mnist_labels, dtype=torch.float32)
    # Create a PyTorch dataset using TensorDataset
    custom_dataset = TensorDataset(images_tensor, labels_tensor)
    # Define the data loader
    train_loader = torch.utils.data.DataLoader(
        custom_dataset, batch_size=4, shuffle=True
    )

    # Define the neural network class
    class MLP(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(784, 1024)
            self.fc2 = nn.Linear(1024, 1024)
            self.fc3 = nn.Linear(1024, 10)

        def forward(self, x):
            x = x.view(-1, 784)
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = torch.log_softmax(self.fc3(x), dim=1)
            return x

    # Define the model, optimizer, and loss function
    model = MLP()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # Function to calculate accuracy
    def accuracy(model, data_loader):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in data_loader:
                inputs, labels = data
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
        return correct / total

    # Train the model
    num_epochs = 20
    train_accs = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for _, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(
            f"Epoch {epoch + 1}, Loss: {(running_loss / len(train_loader)):.4f}",
            end=". ",
        )
        # Calculate accuracy on the training set
        train_accuracy = accuracy(model, train_loader)
        train_accs.append(train_accuracy)
        print(f"Training set accuracy: {train_accuracy}%")

    # Get model parameters
    params = model.state_dict()

    # Return training accuracy and model parameters
    return train_accs, params

In [ ]:
train_accs, params = mnist_3_linear_layers_torch(
    mnist_images=mock_images, mnist_labels=mock_labels
)

#### Inspect the training accuracies and the model's parameters

In [ ]:
train_accs

In [ ]:
assert isinstance(params, dict)

## 3. Now that the code works on mock data, the DS submits the code request for execution to the DO

#### First the DS wraps the training function with the `@sy.syft_function` decorator

In [ ]:
@sy.syft_function(
    input_policy=sy.ExactMatch(
        mnist_images=mock_images_ptr, mnist_labels=mock_labels_ptr
    ),
    output_policy=sy.SingleExecutionExactOutput(),
)
def mnist_3_linear_layers_torch(mnist_images, mnist_labels):
    # third party
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset

    # Convert NumPy arrays to PyTorch tensors
    images_tensor = torch.tensor(mnist_images, dtype=torch.float32)
    labels_tensor = torch.tensor(mnist_labels, dtype=torch.float32)
    # Create a PyTorch dataset using TensorDataset
    custom_dataset = TensorDataset(images_tensor, labels_tensor)
    # Define the data loader
    train_loader = torch.utils.data.DataLoader(
        custom_dataset, batch_size=4, shuffle=True
    )

    # Define the neural network class
    class MLP(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(784, 1024)
            self.fc2 = nn.Linear(1024, 1024)
            self.fc3 = nn.Linear(1024, 10)

        def forward(self, x):
            x = x.view(-1, 784)
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = torch.log_softmax(self.fc3(x), dim=1)
            return x

    # Define the model, optimizer, and loss function
    model = MLP()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # Function to calculate accuracy
    def accuracy(model, data_loader):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in data_loader:
                inputs, labels = data
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
        return correct / total

    # Train the model
    num_epochs = 20
    train_accs = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for _, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(
            f"Epoch {epoch + 1}, Loss: {(running_loss / len(train_loader)):.4f}",
            end=". ",
        )
        # Calculate accuracy on the training set
        train_accuracy = accuracy(model, train_loader)
        train_accs.append(train_accuracy)
        print(f"Training set accuracy: {train_accuracy}%")

    # Get model parameters
    params = model.state_dict()

    # Return training accuracy and model parameters
    return train_accs, params

#### Then the DS creates a new project with relevant name and description, as well as specify itself as a member of the project

In [ ]:
new_project = sy.Project(
    name="Training a 3-layer torch neural network on MNIST data",
    description="""Hi, I would like to train my neural network on your MNIST data 
                (I can download it online too but I just want to use Syft coz it's cool)""",
    members=[ds_client],
)
new_project

#### Add a code request to the project

In [ ]:
new_project.create_code_request(obj=mnist_3_linear_layers_torch, client=ds_client)

In [ ]:
ds_client.code

#### Start the project which will notifies the DO

In [ ]:
project = new_project.send()

In [ ]:
assert isinstance(project, sy.service.project.project.Project)

In [ ]:
project.events

In [ ]:
project.requests

In [ ]:
project.requests[0]

### 📓 Now switch to the [second DO's notebook](./02-data-owner-review-approve-code.ipynb)